# Classificação

A tarefa de **classificação** consiste em predizer **rótulos** para determinados **objetos**. Vamos contextualizar de maneira operacional esses objetos e rótulos através de seus tipos de variáveis: em geral um rótulo é um número inteiro <code>(int)</code> ou uma string <code>(str)</code>; já o objeto a ser classificado pode ser uma imagem, um áudio, ou, de maneira mais geral, um **vetor de features** - aqui vetor tem o mesmo sentido de uma variável lista <code>(lst)</code> ou uma tupla <code>(tuple)</code>. 

Diferentes **algoritmos de classificação** podem ser usados para realizar essa tarefa de, dada uma entrada (objeto a ser classificado), gerar uma saída (rótulo predito para aquele objeto).

Define-se a classificação como uma tarefa de **aprendizado de máquina supervisionado**, isto é, os algoritmos de classificação precisam ser treinados com pares objeto-rótulo tidos como associações corretas. Somente após o treinamento é que o algoritmo de classificação estará apto a predizer rótulos para novas entradas (das quais não se sabe o rótulo a princípio) - diz-se que  algoritmo aprendeu o padrão nos dados de treinamento e agora pode ser usado para classificar novas entradas.

-----------------------------------------------

In [ ]:
# No google colab é preciso atualizar a versão do matplotlib para gerar alguns detalhes nos gráficos
#!pip install matplotlib --upgrade 

## 1 - MAGIC Gamma Telescope Data Set

#### Descrição geral:
O problema consiste na detecção de raios gama primários usando um Telescópio de Radiação Cherenkov baseado em solo.
Os dados foram gerados por Monte Carlo (com parâmetros para gerar energias menores do que 50 GeV), simulando registros de eventos de raios gama e background. Os registros são imagens de chuveiros de partículas (shower image) que permitem distinguir entre raios gama (sinal alvo) ou imagens de chuveiros hadronicos iniciados por partículas cósmicas no topo da atmosfera (background). A imagens foram pré-processadas para gerar atributos (features) que as caracterizam.

#### Objetivo:
Prover um meio de software para auxiliar na detecção de partículas gama. O modelo que iremos treinar pode ser usado como um software de um instrumento de medida ou um software para auxiliar na análise de um grande volume de dados.


#### Features (variáveis de entrada):
As features foram extraídas de processamentos feitos sobre as imagens. São elas:
- **fLength**: major axis of ellipse [mm]
- **fWidth**: minor axis of ellipse [mm]
- **fSize**: 10-log of sum of content of all pixels [in #phot]
- **fConc**: ratio of sum of two highest pixels over fSize [ratio]
- **fConc1**: ratio of highest pixel over fSize [ratio]
- **fAsym**: distance from highest pixel to center, projected onto major axis [mm]
- **fM3Long**: 3rd root of third moment along major axis [mm]
- **fM3Trans**: 3rd root of third moment along minor axis [mm]
- **fAlpha**: angle of major axis with vector to origin [deg]
- **fDist**: distance from origin to center of ellipse [mm]

#### Classes (rótulos de saída):
- g = gamma (signal)
- h = hadron (background) 

#### Referências:
Diversos artigos foram públicads baseados nesses dados. Vamos nos ater a dois repositórios de dados secundários e ao artigo original:
- https://archive.ics.uci.edu/ml/datasets/magic+gamma+telescope
- https://www.openml.org/search?type=data&sort=runs&id=1120&status=active
- Bock, R.K., Chilingarian, A., Gaug, M., Hakl, F., Hengstebeck, T., Jirina, M., Klaschka, J., Kotrc, E., Savicky, P., Towers, S., Vaicilius, A., Wittek W. (2004). Methods for multidimensional event classification: a case study using images from a Cherenkov gamma-ray telescope. Nucl.Instr.Meth. A, 516, pp. 511-528 (https://www.sciencedirect.com/science/article/abs/pii/S0168900203025051?via%3Dihub)
- Uma imagem de telescópio Cherenkov pode ser vista aqui https://arxiv.org/pdf/2110.14527.pdf.

### 1 - Primeiro passo: entendendo os dados (Análise Exploratória)

Importando as bibliotecas que usaremos:

In [ ]:
from sklearn.datasets import fetch_openml # Para importar os dados do site OpenML
import pandas as pd                       # Para trabalhar com tabelas
import seaborn as sns                     # Para gerar gráficos
import matplotlib.pyplot as plt           # Para gerar gráficos

Buscando os dados no repositório online (OpenML):

In [ ]:
dados = fetch_openml(data_id=1120)  # Estamos passando o id dos dados e salvando o resultado na variável dados

In [ ]:
type(dados) # A variável dados é do tipo Bunch

In [ ]:
dados

In [ ]:
dados.keys() # Aqui estão as chaves (campos) contidas na variável dados

In [ ]:
type(dados.data) # Dentro do campo 'data' da váriavel dados, temos um objeto tipo DataFrame

In [ ]:
dados.data.head() # Por ser um objeto tipo DataFrame ele possui os métodos de um dataframe como o .head()

In [ ]:
type(dados.target)

In [ ]:
dados.target # Aqui temos as classes alvo de cada detecção

Vamos juntar todas as informações em uma única tabela (DataFrame) para facilitar as análises subsequentes.

In [ ]:
df = dados.data.copy()  # Copiando os dados em uma variável df (o copy gera um novo um objeto e não só uma referência)
df['class'] = dados.target.copy() # Criando uma coluna que recebe o campo target da variável dados

In [ ]:
df.head()

Agora vamos gerar estatísticas básicas dados dados:

In [ ]:
df.info() # Nos diz informações gerais sobre a tabela

In [ ]:
df.describe() # Nos diz estatísticas básicas de cada coluna

In [ ]:
df.groupby('class').mean() # Média de cada feature agrupada pela classe

In [ ]:
df.groupby('class').std() # Desvio padrão de cada feature agrupada pela classe

In [ ]:
# Gerando dispersões par a par de todas as variáveis, e distribuições de cada variável, coloridas pela classe
# Pode demorar alguns minutos para executar
# sns.pairplot(df, hue='class')

Calculando a correlação entre cada par de variáveis de entrada:

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(df.corr(method = 'pearson'), annot=True, fmt=".1f");

### 1 - Segundo passo: separar os dados

Para o aprendizado supervisionado queremos separar os dados entre dados de entrada (features) e dados de saída (classe alvo), mas também entre **dados de treinamento** e **dados de teste**. Os dados de treinamento são fornecidos aos pares (entrada/saída) para o algoritmos aprender o padrão. Com os dados de teste iremos fornecer apenas as entradas e comparar as saídas preditas pelo algoritmo treinado com as saídas que já sabiamos - isso nos dirá quanto que o algoritmo está acertando.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Escolhendo as colunas de entrada x e as colunas de saída y
x = df.drop(columns = ['class']) # Estamos dropando (jogando fora) a coluna class (perceba que é a saida)
y = df['class'] # Classe alvo

# Dividindo conjunto de treinamento e conjunto de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 8, stratify=y)

# O parâmetro stratify mantem a mesma proporção de exemplos para cada classe
# O parâmetro test_size determinado quantos porcento dos dados serão usados para teste
# O parâmetro random_state é um seed de número pseudoaleatório usado para gerar a sequência de linhas das tabela 
# que será de treinamento e aquelas que serão de teste, i.e., estamos separando a tabela de forma aleatória

In [ ]:
len(x_train), len(y_train), len(x_test), len(y_test) # Vendo a quantidade de linhas em cada conjunto

### 1 - Terceiro passo: transformar os dados 

Alguns algoritmos de aprendizado de máquina funcionam melhor quando cada uma das features de entrada está na mesma escala. Por exemplo, digamos que tenhamos duas colunas de entrada sobre duas medidas importantes para o nosso problema, mas uma medida é feita em km e a outra em mm (uma diferença de 6 ordens de magnitude): alguns algoritmos irão estimar que a importância da entrada em km é maior do que da entrada em mm (podendo inclusive desprezar essa entrada). Para evitar isso, podemos adicionar uma etapa de transformação de escalonamento dos dados.

Existem várias formas de escalonar os dados, sendo as duas mais usadas a **Padronização** (standardization) e a **Normalização** (normalization).

Para mais informações sobre tipos de escalonamento, consulte:
https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py

Algoritmos cujo escalonamento dos dados de entrada pode ser importante:
- K-nearest neighbors
- Logistic regression
- SVM
- Perceptrons e redes neurais artificiais

Algoritmos invariantes a escalonamento da entrada:
- Fisher LDA
- Naive Bayes
- Decision trees
- RandomForest
- GradientBoosting

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
# Instanciando o Escalonador
#scaler = StandardScaler()
scaler = MinMaxScaler()

# Treinando o escalonador
scaler.fit(x_train)

# Usando o escalonador treinado para transformar os dados
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

### 1 -  Quarto passo: treinar o algoritmo 

Devemos escolher um algoritmo de classificação (existem vários!).
Consulte https://scikit-learn.org/stable/supervised_learning.html#supervised-learning para saber mais sobre os classificadores disponíveis na biblioteca Sklearn.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Criamos o objeto do classificador (não mudamos nenhum hiperpârametro)
classificador_lr = LogisticRegression()  

# Treinamos o classificador passando apenas o conjunto de dados de treinamento 
classificador_lr.fit(x_train_scaled, y_train) 

### 1 - Quinto passo: testar e avaliar 

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report
from sklearn.metrics import cohen_kappa_score, roc_auc_score, roc_curve

In [ ]:
# Perceba que estamos passando apenas o x de teste, afinal o algoritmo é que nos dira qual é o y 
y_predicoes_lr = classificador_lr.predict(x_test_scaled) 

# Alguns algoritmos soltam, além dos rótulos, um valor de confiança entre 0 e 1 naquele rótulo predito
confidence_lr = classificador_lr.predict_proba(x_test_scaled) 

In [ ]:
y_predicoes_lr

In [ ]:
confidence_lr

In [ ]:
#confidence[::,0] # Pegar apenas o primeiro elemento de cada par de probabilidade
#confidence[::,1] # Pegar apenas o segundo elemento de cada para de probabilidade

A matriz de confusão nos dá uma forma de verificar visualmente como os acertos e erros do classificador estão distribuídos entre as classes.
- True Label é o rótulo verdadeiro que sabiamos de antemão;
- Predicted Label é o rótulo predito pelo classificador (que gostaríamos que fosse igual ao rótulo verdadeiro). 

In [ ]:
matriz_confusao = confusion_matrix(y_true = y_test,
                                   y_pred = y_predicoes_lr,
                                   labels=['g','h'])

# plotando uma figura com a matriz de confusao
figure = plt.figure(figsize=(15, 5))
disp = ConfusionMatrixDisplay(confusion_matrix = matriz_confusao, display_labels=['g','h'])
disp.plot(values_format='d') 
plt.show()

A partir dos quadrantes na matriz de confusão podemos calcular algumas métricas:
- **Precisão** é a quantidade de acertos do classificador para a classe alvo (neste caso, a classe g) levando-se em consideração **tudo que foi predito como dentro do escopo**. É calculado pela divisão $\frac{TP}{TP+FP}$
- **Revocação** é a quantidade de acertos para a classe dentro do escopo levando-se em consideração **tudo que deveria ter sido predido dentro do escopo**. É calculada pela divisão $\frac{TP}{TP+FN}$
- **F1-Score** é a média harmônica entre a precisão e a revocação.
- **Acurácia** é a razão entre a quantidade de acertos e o total de elementos testados $\frac{TP+TN}{TP+TN+FP+FN}$
- **Suporte** é a quantidade de exemplos em cada classe.

In [ ]:
# Metricas de precisão, revocação, f1-score e acurácia.
print(classification_report(y_test, y_predicoes_lr))

In [ ]:
# Métrica do Coeficiente Kappa de Cohen nos diz quão distante o classificador está de um classificador aleatório
cohen_kappa_score(y_test, y_predicoes_lr)

In [ ]:
# pos_label é o label da classe que está no nosso escopo, aquela que queremos descobrir
# Perceba que estamos pegando as probabilidades referentes ao mesmo label
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, confidence_lr[::,0], pos_label='g')

# A integral da curva ROC nos diz quão bom o classificador for em discernir as duas classes
auc_lr = roc_auc_score(y_test, confidence_lr[::,1])

In [ ]:
plt.plot(fpr_lr,tpr_lr)
plt.ylabel('True Positive Rate'), plt.xlabel('False Positive Rate')
plt.plot([0,1],[0,1], linestyle='--', color='grey')   # Reta de probabilidade 1/2 (jogar moeda)
plt.xlim([0,1]), plt.ylim([0,1])                      # Limites do gráfico
plt.text(0.8,0.1,f'AUC={str(round(auc_lr,3))}')       # Valor da integral da ROC
plt.show()

### 1 - Voltando para o passo 4
Como podemos melhorar o classificador?
Várias alternativas são possíveis:
- **Seleção de Features**: escolher quais atributos de entrada usaremos;
- **Pré-processamento das entrada**: os dados tem outliers? Quão bons são os dados usados?
- **Transformações das entrada**: mudar escalonamento ou outras técnicas de transformação;
- **Comparar Algoritmos**: testar outros algoritmos para a mesma tarefa;
- **Ajuste de Hiperparâmetros**: modificar os hiperparâmetros de um algoritmo para melhorar sua performance.

Vamos fazer um teste com uma Random Forest:

In [ ]:
# Primeiro passo: carregar dados
# Não precisamos refazer

# Segundo passo: separar dados
# Não precisamos refazer 

# Terceiro passo: transformar dados
# Não precisamos refazer

# Quarto passo: treinar algoritmo
from sklearn.ensemble import RandomForestClassifier
classificador_rf= RandomForestClassifier() 
classificador_rf.fit(x_train_scaled, y_train)

# Quinto passo: testar
y_predicoes_rf = classificador_rf.predict(x_test_scaled) 
confidence_rf = classificador_rf.predict_proba(x_test_scaled)

In [ ]:
print(classification_report(y_test, y_predicoes_rf))

In [ ]:
cohen_kappa_score(y_test, y_predicoes_rf)

In [ ]:
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, confidence_rf[::,0], pos_label='g')
auc_rf = roc_auc_score(y_test, confidence_rf[::,1])

In [ ]:
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression  AUC_LG={str(round(auc_lr,3))}')
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest         AUC_RF={str(round(auc_rf,3))}')
plt.ylabel('True Positive Rate'), plt.xlabel('False Positive Rate')
plt.plot([0,1],[0,1], linestyle='--', color='grey') # Reta de probabilidade 1/2 (jogar moeda)
plt.xlim([0,1]), plt.ylim([0,1])                    # Limites do gráfico
plt.legend()
plt.show()

O modelo usando algoritmo de RandomForest obteve um desempenho melhor do que o modelo de Regressão Logística!

Podemos retornar os passos para tentar melhorá-lo ainda mais: mas qual seria nossa meta? Em geral, quando estamos comparando desempenho de modelos de Aprendizado de Máquina é uma boa prática ter um **modelo base** como referência, isto é, uma métrica de performance que representa o estado da arte atual ou a situação atual do sistema que estamos melhorando.

Vamos assumir que o desempenho obtido é suficiente para nossos propósitos (convidamos os alunos a explorarem as outras possibilidades de aperfeiçoamento como um exercício), e prosseguir com novos pontos sobre modelos de IA.

### 1 - Sexto passo: uso e deployment do modelo

Uma vez que o modelo está treinado e com uma métrica de desempenho aceitável, podemos aplicá-lo para um objetivo específico.

Por exemplo, podemos testá-lo para uma entrada genérica (lembre-se que as entradas estão escalonadas de acordo com nosso procedimento adotado):

In [ ]:
classificador_lr.predict([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.55]])

Podemos usar o modelo treinado para classificar qualquer nova entrada. Assim, temos uma pergunta operacional para responder agora:

Como podemos usar o modelo treinado sem ter que repetir todos os passos anteriores da próxima vez?

Dentro do código o modelo está alocado em uma variável, por exemplo <code>classificador_rf</code>. Essa variável na verdade é uma estrutura (objeto) na memória RAM do computador. Uma vez que pararmos de rodar nosso código, o modelo treinado se perde (e teremos começar desde o passo um de novo).

Então queremos serializar o modelo, isto é, passá-lo da memória volátil para a memória permanente do computador.

Alguns modelos são paramétricos e possuem uma estrutura matemática explicita, como uma equação. Treinar o modelo, neste caso, significa encontrar os coeficientes dessa equação. Vejamos a Regressão Logística:

### $p(\pmb{x})=\frac{1}{1+e^{-(\pmb{aX}+b)}}$

In [ ]:
# Os coeficientes a estão no atributo .coef_ do objeto classificador devidamente treinado
classificador_lr.coef_

In [ ]:
# O coeficiente linear está no atributo .intercept_ do objeto classificador devidamente treinado
classificador_lr.intercept_

Dessa forma, tendo os coeficientes e equação que rege o algoritmo, podemos reconstruir o modelo a qualquer momento:

In [ ]:
import numpy as np

In [ ]:
def log_reg(x,a,b):
    t = np.inner(x,a)+b
    proba = 1/(1+np.exp(-t))
    
    y_pred = []
    for yi in proba:
        if yi>=0.5:
            y_pred.append('h')
        else:
            y_pred.append('g')
    
    return np.array(y_pred)

In [ ]:
log_reg(x_test_scaled,classificador_lr.coef_,classificador_lr.intercept_)

In [ ]:
y_predicoes_lr

Perceba que isso significa que uma forma de armazenar o modelo de Regressão Logística na memória permanente do computador é definir a equação (método) como feito na função <code>log_reg</code> acima e salvar os valores do coeficientes.

De vez de fazer isso manualmente, podemos utilizar funções prontas que realizam essa tarefa.

Para serializar objetos na memória em Python pudemos utilizar o formato <code>.pickle</code>, como abaixo:

In [ ]:
import pickle

In [ ]:
# vamos salvar em bytes (flag wb) para ser mais cross-platform (acessível a vários sistemas)
with open('meu_modelo_serializado.pickle', 'wb') as f: 
    pickle.dump(classificador_lr, f)

Pronto! Nosso modelo está salvo. Se quisermos usá-lo em um código em Python, podemos simplesmente carregá-lo:

In [ ]:
with open('meu_modelo_serializado.pickle', 'rb') as f:
    modelo_carregado = pickle.load(f)

In [ ]:
modelo_carregado.predict([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.55]])

Além do formato <code>.pickle</code> é possível exportar o modelo treinado usando um módulo da própria biblioteca do ScikitLearn:

In [ ]:
from joblib import dump, load

In [ ]:
dump(classificador_lr, 'meu_modelo_serializado.joblib') 

In [ ]:
modelo_carregado2 = load('meu_modelo_serializado.joblib') 

In [ ]:
modelo_carregado2.predict([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.55]])

A vantagem de salvar o modelo permanentemente em um arquivo <code>.pickle</code> ou <code>.joblib</code> é que ele pode ser exportado para outros sistemas, inclusive disponibilizado para outras pessoas utilizarem. Na comunidade de IA modelos disponibilizados assim são chamados de **modelos pré-treinados**.

Perceba que a persistência feita dessa forma é válida inclusive para modelos não paramétricos como as RandomForest (que precisam armazenar a sequência de perguntas if/else que melhor separam os dados, como uma árvore de decisão - e não coeficientes e um equação como a regressão logística que vimos acima). Outra importante vantagem é que alguns modelos podem levar dias ou semanas para serem treinados: retreiná-los para cada um dos usos não faria sentido!

Além da aplicação dos modelos como módulos de predição (que podem ser incorporados em diferentes fluxos de processamento), podemos utilizar os modelos treinados para extrair algum conhecimento do problema. Essa é uma grande área de pesquisa conhecida como **XAI - AI Explainability**. Vamos demonstrar algumas ideias para os modelos de Regressão Logística e Random Forest:

In [ ]:
# As nossas variaveis de entrada são:
x.columns

In [ ]:
# E os coeficientes do nosso modelo de Regressão Logística são:
classificador_lr.coef_

Como as entradas foram escalonadas, a dimensão delas é comparável. Isso significa que um coeficiente maior (em módulo) para uma variável deve corresponder a uma importância maior daquela variável para o modelo. 

Isso pode nos dar dicas de quais variáveis são mais informativas para o nosso modelo, da mesma forma que uma **análise dimensional** permite dizer se a relação das unidades físicas das variáveis correspondem a função matemática estabelecida entre elas.

Observe, contudo, que algumas variáveis estão correlacionadas. Isso precisa ser levado em conta na hora de se selecionar as variáveis de entrada e considerar suas importância através do "tamanho" dos seus coeficientes. Falaremos mais sobre isso na aula de **redução de dimensionalidade**.

Perceba que, a **Regressão Logística é uma método paramétrico** por isso temos acesso aos coeficientes (temos um modelo matemático bem definido para relacionar entradas e saídas). Por outro lado, uma Árvore de Decisão e métodos de ensemble baseados em árvores (como Random Forest e Gradient Boosting) são não paramétricos, ou seja, não possuem coeficientes do modelo que podemos associar claramente a cada entrada (OBS: redes neurais, apesar de paramétricos podem ser de dificil associação, já que, em geral, possuem mais coeficientes do que entradas).

Contudo, é possível estabelecer uma medida de importância para cada feature baseada em como as árvores dividem os exemplos fornecidos. Essas medidas são chamadas de **medidas de impureza**. 

In [ ]:
classificador_rf.feature_importances_ # Método de impureza usando coeficiente de Gini

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,4))
sns.barplot(x=x.columns, y=classificador_lr.coef_[0], ax=ax[0], palette='hls')
ax[0].tick_params(axis='x', rotation=90), ax[0].grid(axis='y'), ax[0].set_title('Reg Logistica')
sns.barplot(x=x.columns, y=classificador_rf.feature_importances_, ax=ax[1], palette='hls')
ax[1].tick_params(axis='x', rotation=90), ax[1].grid(axis='y'), ax[1].set_title('Random Forest')

Observações adicionais:
- Outra forma de medir importância de feature é através da **permutação de features**;
- Métodos automáticos de **seleção de features** são também conhecidos como **métodos wrapper**;
- O momento exato (os passos aqui referenciados) para se fazer a seleção de features pode depender de metodologias mais robustas, como aquelas que usam diferentes formas de **Validação Cruzada** (veja a discussão em Krstajic et. al., https://jcheminf.biomedcentral.com/articles/10.1186/1758-2946-6-10).

---------------------------------------

## 2 - IArpi Data Set

#### Descrição geral:
Três diferentes objetos são postos a se mover em um plano inclinado devido a ação da gravidade. Atributos cinemáticos do movimento dos corpos são coletados. Pretende-se estabelecer uma relação entre esses atributos e cada tipo de objeto.

#### Objetivo:
O problema consiste na classificação de três objetos (esfera, cilindro e aro) a partir de atributos cinemáticos do seu movimento em um plano inclinado. O objetivo é introduzir técnicas de IA para cursos de graduação de física onde o experimento do plano inclinado é amplamente estudado (de maneira teórica e em laboratório).


#### Features (variáveis de entrada):
As features foram determinadas experimentalmente:
- Ângulo: ângulo de inclinação do plano [°]
- Distância:  distância percorrida pelo objeto [m]
- Altura: de partida do objeto [m]
- Tempo: intervalo de tempo para percorrer a distância [s]
- Velocidade Média: velocidade média determinada pela distância/tempo [m/s]

#### Classes (rótulos de saída):
- esfera
- cilindro
- aro

#### Referências:
- https://github.com/simcomat/IArpi
- Ferreira, H., Almeida Junior, E. F., Espinosa-García, W., Novais, E., Rodrigues, J. N. B., & Dalpian, G. M. (2022). Introduzindo aprendizado de máquina em cursos de física: o caso do rolamento no plano inclinado. In Revista Brasileira de Ensino de Física (Vol. 44). FapUNIFESP (SciELO). https://doi.org/10.1590/1806-9126-rbef-2022-0214 

###  2 - Primeiro passo: entendendo os dados

In [ ]:
import pandas as pd  # Importando a biblioteca pandas com o nome pd
import numpy as np   # Trabalhar com vetores e matrizes de números

from sklearn.model_selection import train_test_split # Separação treino e teste dos dados
from sklearn.preprocessing import MinMaxScaler       # Escalonador

# Diferentes algoritmos supervisionados de classificação
from sklearn.dummy import DummyClassifier           # Modelo base
from sklearn.neighbors import KNeighborsClassifier  # k-vizinhos mais próximos (KNN)
from sklearn.ensemble import RandomForestClassifier # RandomForest
from sklearn.ensemble import GradientBoostingClassifier # GradientBoosting
from sklearn.svm import SVC                         # Maquina de Vetor Suporte SVM
from sklearn.neural_network import MLPClassifier    # Multlayer Perceptron
from sklearn.naive_bayes import GaussianNB          # Naive Bayes
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis

# Métricas de desempenho
from sklearn.metrics import accuracy_score, cohen_kappa_score, make_scorer  # Métricas de  Classificacao
from sklearn.metrics import confusion_matrix                                # Métricas de Classificacao

# Gráficos
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.transforms as mtransforms
from matplotlib.ticker import AutoMinorLocator
from matplotlib.lines import Line2D

Vamos baixar os dados direto do Github do projeto:

In [ ]:
tabela_dados = pd.read_csv('https://raw.githubusercontent.com/simcomat/IArpi/main/datasets/rolling.csv', sep=';') 

In [ ]:
tabela_dados.head()

In [ ]:
tabela_dados.info()

### 2 - Segundo passo: separar os dados

In [ ]:
# Escolhendo o que é entrada e o que é saída
x = tabela_dados[['Altura (m)','Ângulo (°)', 'Tempo (s)']]  # Features
y = tabela_dados['Objeto']                                  # Atributo alvo

# Dividindo conjunto de treinamento e conjunto de teste
# Stratify garante que a quantidade de cada objeto seja aproximadamente o mesmo nos conjuntos de treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 10, stratify=y)

### 2 - Terceiro passo: transformar os dados de entrada

In [ ]:
scaler = MinMaxScaler()   # Instanciando o escalonador
scaler.fit(x_train)       # Treinando o escalonador apenas com os dados de treinamento

x_train_scaled = scaler.transform(x_train)   # Transformando os dados de treinamento pelo escalonador treinado
x_test_scaled = scaler.transform(x_test)     # Transformando os dados de teste pelo escalonado treinado

### 2 - Quarto passo: treinar o algoritmo

Neste caso iremos treinar vários algoritmos diferentes e observar a performance de cada um:

In [ ]:
# Dummy Classifier (Modelo base)
base_model = DummyClassifier(strategy="uniform")
base_model.fit(x_train_scaled,y_train)

# LDA (Discriminante Linear)
lda = LinearDiscriminantAnalysis()  # Criando classificador (sem nenhum hiperparametro)
lda.fit(x_train_scaled, y_train)    # Treinamos o classificador passando apenas o conjunto de dados de treinamento 

# QDA (Discriminante Quadrático)
qda = QuadraticDiscriminantAnalysis()
qda.fit(x_train_scaled, y_train)

# GaussianNB
gNB = GaussianNB()
gNB.fit(x_train_scaled, y_train)

# KNN
knn = KNeighborsClassifier()     # Criando classificador (sem nenhum hiperparametro)
knn.fit(x_train_scaled, y_train) # Treinamos o classificador passando apenas o conjunto de dados de treinamento 

# SVM
svm = SVC()
svm.fit(x_train_scaled, y_train)

# MLP 
mlpc = MLPClassifier(random_state=42)
mlpc.fit(x_train_scaled, y_train)
# (As iterações de aprendizado podem alcançar o limite default emitindo um warning) 

# RandomForest
rf = RandomForestClassifier(random_state=42) # Criando classificador (hiperparametro de seed)
rf.fit(x_train_scaled, y_train) #  

#Gradient Boosting
gboo = GradientBoostingClassifier(random_state=42)
gboo.fit(x_train_scaled, y_train)

Vamos criar um discionário contendo todos os objetos dos classificadores treinados. Dessa forma, poderemos acessar qualquer um dos modelos através da variável ``` classificacores```:

In [ ]:
classificadores = {
    'BM':base_model,
    'LDA':lda,
    'QDA':qda,
    'GNB':gNB,
    'KNN':knn,
    'SVM':svm,
    'RF':rf,
    'GB':gboo,
    'MLP':mlpc
}

Agora vamos criar variáveis (estruturas dicionário e lista) para armazenar o valor de cada métrica calculada para cada modelo testado.

In [ ]:
resultados={}
resultados_kappa={}
resultados_accuracia={}

lab = ['esfera','cilindro','aro']
for clf_name, clf in classificadores.items():  # Iterando sobre todos os modelos treinados
    y_pred = clf.predict(x_test_scaled)        # Passando para o ML apenas os dados de teste escalonados
    
    acc = accuracy_score(y_test, y_pred)
    kappa =  cohen_kappa_score(y_test, y_pred, labels=lab)
    
    scoring = {'accuracy': acc,
               'kappa' :kappa
         }
    resultados_kappa[clf_name]=kappa
    resultados_accuracia[clf_name]=acc
    resultados[clf_name]=scoring

Vamos usar uma estrutura de dados pandas DataFrame para visualizar os dados das métricas que armazenamos no passo anterior:

In [ ]:
resultado_teste_classificao = pd.DataFrame(data=resultados)
resultado_teste_classificao.head()

### 2 - Comparando com o Modelo Físico

Para corpos com simetria radial e massa uniformemente distribuída pode o momento de inércia pode ser expresso por $\beta mR^2$, onde $m$ é a massa do objeto e $R$ o seu raio. 

Além disso, considerando que o objeto rola sem deslizar sobre o plano (i.e. há um vínculo entre o movimento de translação do centro de massa e um ponto na superfície de contado do objeto com o plano), então podemos expressar a cinemática desse problema através da velocidade média $V_{med}$ do objeto solto a uma altura $h$.

In [ ]:
# A função recebe a altura, tempo e o ângulo de inclinação e devolve beta predito
def encontra_beta(altura, tempo, theta):
    g=9.8                                        # Aceleração da gravidade
    distancia=altura/np.sin(np.deg2rad(theta))   # Distância percorrida sobre o plano
    vmed = distancia/tempo                       # Velocidade média do objeto
    beta = (0.5*g*altura/vmed**2)-1
    return beta

In [ ]:
tabela_dados['Beta MF'] = tabela_dados.apply(lambda x: encontra_beta(x['Altura (m)'],
                                                                     x['Tempo (s)'],
                                                                     x['Ângulo (°)']), axis=1)

In [ ]:
tabela_dados.head(3)

Para comparar, vamos criar uma imagem. Perceba que o modelo físico prevê um valor contínuo e não uma classe. Podemos utilizar uma análise gráfica da distribuição de valores previstos 

In [ ]:
y_pred= knn.predict(x_test_scaled)                # Resultados apenas do KNN
matriz_confusao=confusion_matrix(y_test, y_pred)  # Matriz de confusão do KNN
result_test=tabela_dados.iloc[x_test.index]       # Pegando apenas as linhas de teste da tabela original (para ver o MF)

In [ ]:
# Formatação da imagem
# Definição dos tamanhos de fontes e ticks dos gráficos
fsize = 12
tsize = 10
major = 5.0
minor = 3.0

style = 'default'
plt.style.use(style)

#plt.rcParams['text.usetex'] = True  # Para usar fonte tex (precisa instalar o tex antes)
plt.rcParams['font.size'] = fsize      
plt.rcParams['legend.fontsize'] = tsize
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = major
plt.rcParams['ytick.minor.size'] = minor

In [ ]:
# Figura Classificação
fig, axd = plt.subplots(1,2, figsize=(8, 3.5) )


# Matriz de Confusao
sns.heatmap(matriz_confusao, annot=True, ax=axd[0], cmap="YlGn",
            xticklabels=['esfera', 'cilindro', 'aro'], yticklabels=['esfera', 'cilindro', 'aro'],
            cbar_kws={'label': 'Quantidade de Exemplos Testados'}, robust=True)
axd[0].set_ylabel('Objeto Verdadeiro')
axd[0].set_xlabel('Objeto Predito')
axd[0].set_title('Aprendizado de Máquina')

  
# Violinplot
sns.violinplot(y='Objeto',x='Beta MF', data=result_test, order =['esfera', 'cilindro', 'aro'],
               palette="YlGn", ax=axd[1])
axd[1].get_yaxis().set_visible(False)
plt.setp(axd[1].get_yticklabels(), visible=False)
axd[1].set_xlabel('$\\beta$ predito'), axd[1].set_xlim([-0.5,2])
axd[1].set_title('Modelo Físico')
axd[1].xaxis.set_minor_locator(AutoMinorLocator())
axd[1].set_ylim([2.5,-0.5])
axd[1].plot([0.4,0.4], [-1,3], color='#d5e6ac', linestyle='dashed', linewidth = 1) 
axd[1].plot([0.5,0.5], [-1,3], color='#81bc82', linestyle='dashed', linewidth = 1) 
axd[1].plot([1,1], [-1,3], color='#2e7748', linestyle='dashed', linewidth = 1) 
custom_lines = [Line2D([0], [0], color='#d5e6ac', lw=5),
                Line2D([0], [0], color='#81bc82', lw=5),
                Line2D([0], [0], color='#2e7748', lw=5)]
axd[1].legend(handles=custom_lines, labels=['esfera', 'cilindro', 'aro'],
                  title='Objeto Verdadeiro',loc='upper right', frameon=False)

# Escrevendo os itens (a), (b), ... em cada um dos gráficos da figura
labels_subplots=['(a)','(b)']
for i in range(0,2):
    if i==0:
        trans = mtransforms.ScaledTranslation(-20/72, 7/72, fig.dpi_scale_trans)
        axd[i].text(0.0, 1.0, labels_subplots[i], transform=axd[i].transAxes + trans,
                fontsize='medium', verticalalignment='top')
    else:
        trans = mtransforms.ScaledTranslation(10/72, -5/72, fig.dpi_scale_trans)
        axd[i].text(0.0, 1.0, labels_subplots[i], transform=axd[i].transAxes + trans,
                fontsize='medium', verticalalignment='top')


In [ ]:
sns.histplot(data = tabela_dados, x= 'Beta MF', kde=True, hue='Objeto',
            palette=['#d5e6ac','#2e7748','#81bc82'])

### 2 - Validação Cruzada

Uma técnica mais avançada de Aprendizado de Máquina é a validação cruzada. Ela tem diversas aplicações e consiste em usar o dataset de maneira alternada, com cada participação servindo para treino e teste em um determinado momento.

No nosso caso usaremos essa metodlogia para calcular a **variabilidade do desempenho** dos algoritmos com relação ao conjunto de dados fornecido para **treinamento**.

Por hipótese inicial, a distribuição dos dados de treinamento e teste deveriam ser as mesmas (representar amostras da mesma população). Por isso selecionamos de maneira aleatória cada amostra ao separar treino e teste (evitar ordenamento dos valores, por exemplo). Entretanto nossos dados podem representar dados de duas populações distintas (sob a ótica de algum critério). Se essas distribuições forem muito diferentes, treinar com uma delas e testar com a outra geraria resultados ruins. Esse problema é conhecido como **OOD** - out of (train) distribution.

Assim, se nossa **variabilidade** durante uma validação cruzada for alta, então estamos usando populações muito distintas em determinados momentos dos nossos testes (ou treinamento).

**OBS1:** é importante destacar que a Validação Cruzada tem outra aplicação bastante estabelecida na comunidade de Aprendizado de Máquina como metodologia para otimização de hiperparâmetros dos algoritmos. Aqui decidimos introduzir o tópico através de um exemplo mais simples. Para o caso da **otimização de hiperparâmetros**, a validação cruzada garante que o algoritmo treinado não terá visto uma parte do conjunto de dados em nenhum momento do treinamento, permitindo que seja realizado um ajuste fino da performance de cada algoritmo através da escolha dos melhores hiperpârametros de cada um. Uma discussão adicional pode ser obtida aqui https://scikit-learn.org/stable/modules/cross_validation.html e aqui https://jcheminf.biomedcentral.com/articles/10.1186/1758-2946-6-10.

**OBS2:** existem situações onde é preciso selecionar amostras ordenadas de vez de amostras aleatórias, por exemplo, para se fazer a predição do próximo valor em uma série temporal. Quando selecionamos dados aleatórios podemos incorrer em um problema conhecido como **Data Leakage**.

In [ ]:
from sklearn.model_selection import StratifiedKFold  # Para separar os dados em k folds na classificação
from sklearn.model_selection import cross_validate   # Para rodar treinamento e teste sobre kfolds

from sklearn import preprocessing          # Auxilia na transformação dos dados (passo 3)
from sklearn.pipeline import make_pipeline # Permite realizar uma sequência de processamentos

In [ ]:
# Separando os dados de entrada e saída para a Classificação
x = tabela_dados[['Altura (m)','Ângulo (°)', 'Tempo (s)']]  # Features
y = tabela_dados['Objeto']     

# Passo 3, 4 e 5 usando PIPELINE
#Instanciando os algoritmos
base_model = DummyClassifier(strategy="uniform")   # Dummy Classifier
lda = LinearDiscriminantAnalysis()                 # LDA (Discriminante Linear)
qda = QuadraticDiscriminantAnalysis()              # QDA (Discriminante Quadrático)
gNB = GaussianNB()                                 # GaussianNB
knn = KNeighborsClassifier()                       # KNN
svm = SVC()                                        # SVM
mlpc = MLPClassifier(random_state=42)              # MultiLayer Percpetron 
rf = RandomForestClassifier(random_state=42)       # RandomForest
gboo = GradientBoostingClassifier(random_state=42) # Gradient Boosting

# Estamos setando um pipeline que envolve escalonar os dados usando MinMax e depois treinar o algoritmo passado
classificadores = {
    'BM':make_pipeline(preprocessing.MinMaxScaler(), base_model),
    'LDA':make_pipeline(preprocessing.MinMaxScaler(), lda),
    'QDA':make_pipeline(preprocessing.MinMaxScaler(), qda),
    'GNB':make_pipeline(preprocessing.MinMaxScaler(), gNB),
    'KNN':make_pipeline(preprocessing.MinMaxScaler(), knn),
    'SVM':make_pipeline(preprocessing.MinMaxScaler(), svm),
    'RF':make_pipeline(preprocessing.MinMaxScaler(), rf),
    'GB':make_pipeline(preprocessing.MinMaxScaler(), gboo),
    'MLP':make_pipeline(preprocessing.MinMaxScaler(), mlpc)
}

# Setando as métricas de desempenho
scoring = {'accuracy': make_scorer(accuracy_score),
           'kappa': make_scorer(cohen_kappa_score)}

# Rodandos as validações cruzadas
results = [] 
for clf_name, clf in classificadores.items():
    
    # Separando 5 folds garantimos usar 20% dos dados para teste e 80% para treinamento
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    tmp = cross_validate(clf, x, y, cv=cv, scoring=scoring)   
    tmp['clf'] = clf_name
    
    results.append(tmp) 
    
# Organizando os resultados
# Calculando a média e o desvio padrão de cada métrica
ob={
    'classificador':[],
    'accuracy':[],
    'accuracy_std':[],
    'kappa':[],
    'kappa_std':[]
}
for i in range(0, len(results)):
    ob['classificador'].append(results[i]['clf'])
    ob['accuracy'].append(np.mean(results[i]['test_accuracy']))
    ob['accuracy_std'].append(np.std(results[i]['test_accuracy']))
    ob['kappa'].append(np.mean(results[i]['test_kappa']))
    ob['kappa_std'].append(np.std(results[i]['test_kappa']))

df_class = pd.DataFrame(data= ob)

In [ ]:
df_class.head(10)

Fazendo uma figura para comparar os resultados visualmente:

In [ ]:
# Figura Validação Cruzada Classificação e Regressão
fig, axd = plt.subplots(2,1, figsize=(6, 6), sharex=True)

# Gráficos de barras de comparação dos métodos
sns.barplot(x='classificador',y = 'accuracy', data=df_class,
            ax=axd[0], color='#81BC82', edgecolor='grey' )
sns.barplot(x='classificador',y = 'kappa', data=df_class,
            ax=axd[1], color='#81BC82', edgecolor='grey' )

axd[0].set_xlabel(''),
axd[1].set_xlabel('Algoritmos de Classificação')
axd[0].set_ylabel('Acurácia'), axd[0].set_ylim([0,1])
axd[1].set_ylabel('$\kappa$ de Cohen'), axd[1].set_ylim([0,1])
axd[0].set_yticks([0, 0.5, 1])
axd[1].set_yticks([0, 0.5, 1])
axd[0].bar_label(axd[0].containers[0], rotation=90, label_type='center', color='w', fmt='%.2f')
axd[1].bar_label(axd[1].containers[0], rotation=45, label_type='edge', color='k', fmt='%.2f')

# Barras verticais indicando variabilidade pelo desvio padraõ
x_coords = [p.get_x() + 0.5 * p.get_width() for p in axd[0].patches]
y_coords = [p.get_height() for p in axd[0].patches]
axd[0].errorbar(x=x_coords, y=y_coords, yerr=df_class['accuracy_std'], fmt="none", c="r", capsize=0.1)

x_coords = [p.get_x() + 0.5 * p.get_width() for p in axd[1].patches]
y_coords = [p.get_height() for p in axd[1].patches]
axd[1].errorbar(x=x_coords, y=y_coords, yerr=df_class['kappa_std'], fmt="none", c="r", capsize=0.1)


plt.setp(axd[0].get_xticklabels(), visible=False)
axd[1].tick_params(axis='x', rotation=45)

# Escrevendo os itens (a), (b)
labels_subplots=['(a)','(b)']
for i in range(0,2):
    trans = mtransforms.ScaledTranslation(10/72, -5/72, fig.dpi_scale_trans)
    axd[i].text(0.0, 1.0, labels_subplots[i], transform=axd[i].transAxes + trans,
            fontsize='medium', verticalalignment='top')

--------------------------

## Exercícios propostos

1) Reproduzir o classificador de ordem magnética (ferromagnético, antiferromagnético) reportado em Acosta et al. 2022, https://pubs.acs.org/doi/10.1021/acsami.1c21558.

2) Reproduzir o classificador de faixa de bandgap para perovskitas duplas reportado em Wang et al. 2022,   https://pubs.acs.org/doi/abs/10.1021/acsami.1c18477.